In [ ]:
#data format library
import h5py

#numpy
import numpy as np
import pandas as pd
import numpy.ma as ma
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib as mpl
new_rc_params = {'text.usetex': False,
"svg.fonttype": 'none'
}
mpl.rcParams.update(new_rc_params)
plt.rcParams["font.family"] = "Times New Roman"
# %matplotlib notebook
import sys
sys.path.append('/Users/gautam.sridhar/Documents/Repos/Markov_Fish/utils/')
import matplotlib.colors as pltcolors
import os
import copy
import clustering_methods as cl
import operator_calculations as op_calc
import delay_embedding as embed
import stats
import time

np.random.seed(42)

import importlib
importlib.reload(op_calc)

In [ ]:
## Load symbolic sequences

path_to_filtered_data = '/Users/gautam.sridhar/Documents/Repos/Markov_Fish/Datasets/JM_Data/'
f = h5py.File(path_to_filtered_data + 'pool_ex8_PCs/kmeans_labels_K5_N1200_s8684.h5')
lengths_all = np.array(f['MetaData/lengths_data'], dtype=int)
labels_fish_allrec = ma.array(f['labels_fish'],dtype=int)
state_trajs = ma.array(f['state_trajs'])
f.close()

# lengths_all = np.load('/Users/gautam.sridhar/Documents/Repos/ZebraBouts/Datasets/Full_Data/lengths_ex2_recordings.npy')
# lengths_all = lengths

In [ ]:
to_mask = 1300
maxL = np.max(lengths_all)

labels_fish_allrec[labels_fish_allrec == to_mask] = ma.masked
labels_fish = labels_fish_allrec

lengths_rem = lengths_all

In [ ]:
## Select Dataset
np.random.seed(42)
seeds = np.random.randint(0,10000,100)
delay_range = np.arange(1,20,1)
dt = 1
div= 463
n_modes=50
labels_all= ma.concatenate(labels_fish,axis=0)
print(labels_fish.shape)

In [ ]:
P_ensemble = np.load('/Users/gautam.sridhar/Documents/Repos/ZebraBouts/Datasets/Full_Data/P_ensemble_ex8_N1200_s8684.npy')

In [ ]:
from scipy.sparse import diags,identity,coo_matrix, csr_matrix
P_ensemble = csr_matrix(P_ensemble)

In [ ]:
import msmtools.estimation as msm_estimation
delay = 3
dt = 1
print(delay)
# lcs_ensemble,P_ensemble = op_calc.transition_matrix(labels_all,delay,return_connected=True)
lcs_ensemble = msm_estimation.largest_connected_set(P_ensemble)
inv_measure = op_calc.stationary_distribution(P_ensemble)
final_labels = op_calc.get_connected_labels(labels_all,lcs_ensemble)
R = op_calc.get_reversible_transition_matrix(P_ensemble)
eigvals,eigvecs = op_calc.sorted_spectrum(R,k=10,seed=123)
sorted_indices = np.argsort(eigvals.real)[::-1]
eigvals = eigvals[sorted_indices][1:].real
eigvals[np.abs(eigvals-1)<1e-12] = np.nan
eigvals[eigvals<1e-12] = np.nan
t_imp =  -(delay*dt)/np.log(np.abs(eigvals))
eigfunctions = eigvecs.real/np.linalg.norm(eigvecs.real,axis=0)
eigfunctions_traj = ma.array(eigfunctions)[final_labels,:]
eigfunctions_traj[final_labels.mask] = ma.masked

## Simulations at q=2

In [ ]:
st_colors = ['#1F50A3','#D04D49']

In [ ]:
# labels = labels_all

path_to_filtered_data = '/Users/gautam.sridhar/Documents/Repos/ZebraBouts/Results/'
f = h5py.File(path_to_filtered_data + 'pool_ex7_PCs/simlabels_fish_K5_N1200_tau3_cg2.h5')
sims_fishnum = ma.asarray(f['MetaData/simfishes'], dtype=int)
lengths_sims = ma.asarray(f['MetaData/lengths_sims'], dtype=int)
sims_fish = ma.asarray(f['sim_labels'], dtype=int)
f.close()
to_mask = 1300

sims_fish[sims_fish == to_mask] = ma.masked

In [ ]:
kmeans_labels = coh_labels_phi1

In [ ]:
cluster_traj_all = ma.copy(final_labels)
cluster_traj_all[~final_labels.mask] = ma.array(kmeans_labels)[final_labels[~final_labels.mask]]
cluster_traj_all[final_labels.mask] = ma.masked

cluster_fish = cluster_traj_all.reshape(labels_fish.shape[0],labels_fish.shape[1])
cluster_fish_mask = cluster_fish.mask

In [ ]:
lifetimes_fish = [[],[],[],[],[],[],[]]
n_state = 4

for cf in range(cluster_fish.shape[0]):
    cluster_fish[cf,0] = ma.masked
    cluster_fish[cf,lengths_rem[cf]-1] = ma.masked
    data_lifetimes, state_labels = stats.state_lifetime(cluster_fish[cf,::3],3)
    for k,ks in enumerate(state_labels):
        lifetimes_fish[ks].append(data_lifetimes[k])
    
    if len(state_labels) < n_state:
        exc_state_labels = np.delete(np.arange(n_state),state_labels)
        for k,ks in enumerate(exc_state_labels):
            lifetimes_fish[ks].append([0])

In [ ]:
y_errorbar_metastate, x_all_metastate, m_all_metastate, b_all_metastate, tscales_metastate, params_metastate, mean_lifetimes = stats.state_lifetimes_stats(lifetimes_fish, n_states=2, t0=0, tf=1500 ,n_times=100)

In [ ]:
sims_lifetimes_fish = [[],[],[],[],[],[],[]]
n_state = 2
for cf in range(sims_fish.shape[1]):
    print(cf)
    sf = sims_fish[:,cf,:lengths_rem[cf]].squeeze()
    sf[:,0] = ma.masked
    sf[:,-1] = ma.masked
    sf = ma.concatenate(sims_fish[:,cf,:lengths_rem[cf]].squeeze(), axis=0)
    data_lifetimes, state_labels = stats.state_lifetime(sf,3)
    for k,ks in enumerate(state_labels):
        sims_lifetimes_fish[ks].append(data_lifetimes[k])
        
    if len(state_labels) < n_state:
        print(len(state_labels))
        exc_state_labels = np.delete(np.arange(n_state),state_labels)
        for k,ks in enumerate(exc_state_labels):
            sims_lifetimes_fish[ks].append([0])

In [ ]:
y_errorbar_metastate_sims, x_all_metastate_sims, m_all_metastate, b_all_metastate, tscales_metastate, params_metastate, mean_lifetimes = stats.state_lifetimes_stats(sims_lifetimes_fish, n_states=2,t0=0,tf=1500, n_times=100)

In [ ]:
fig,ax = plt.subplots(1,1, figsize=(12.5,12.5))

t0,tf = 1,600

xrange = np.linspace(t0,tf,160)

sim_markers = ['o','s']

# st_names = ['Metastate 1', 'Metastate 2']

# for state_idx in np.unique(kmeans_labels):
#     x,y = stats.cumulative_dist(data_f_lt[state_idx],[0,1000])
#     y = np.array([np.mean(y[x==x_unique]) for x_u nique in np.unique(x)])
#     x = np.sort(np.unique(x))
#     plt.plot(x,1-y,lw=2,ls='--',c=st_colors[state_idx])

for ms in [0,1]:#np.unique(cluster_traj_all.compressed()):

    mean = y_errorbar_metastate[ms][:-1,0]
    cil = y_errorbar_metastate[ms][:-1,1]
    ciu = y_errorbar_metastate[ms][:-1,2]
    
    mean_sim = y_errorbar_metastate_sims[ms][:-1,0]
    cil_sim = y_errorbar_metastate_sims[ms][:-1,1]
    ciu_sim = y_errorbar_metastate_sims[ms][:-1,2]
    
    ax.scatter(x_all_metastate[ms][:-1], mean, marker='^',s=50, color='C{}'.format(ms))#,label = r'Mean Duration$_{} = {}$ Bouts'.format({st_names2[ms][0:2]}, round(mean_lifetimes[ms],2)))
    ax.errorbar(x_all_metastate[ms][:-1], mean,[mean-cil, ciu - mean], fmt='.', capsize=10, color = 'C{}'.format(ms), alpha=0.4)

    ax.scatter(x_all_metastate_sims[ms][:-1], mean_sim,s=100, marker='o',facecolors='none', edgecolors='C{}'.format(ms))
    ax.fill_between(x_all_metastate_sims[ms][:-1], cil_sim, ciu_sim, alpha=0.3, color='C{}'.format(ms))
    
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.set_yscale('log')
# ax.set_title('Actual and Simulated dwell times', fontsize=35)
ax.set_xscale('log')
ax.set_xlabel(r'Number of states',fontsize=40)
ax.set_ylabel(r'Complementary CDF (log scale)',fontsize= 40)
ax.legend(loc="upper left",bbox_to_anchor=(0.4, 0.96), fontsize=30)
plt.xticks(fontsize=30)
plt.yticks(fontsize=30)
# ax.set_xlim(1,1000)
# ax.set_ylim(5e-6,1) 

# plt.savefig('/Users/gautam.sridhar/Documents/ZENITH/Figures/Suppl4/2state_dwelltime_dist.pdf')
plt.show()

In [ ]:
hr_sims = [[],[],[],[],[],[],[]]
sims_ciu = [[],[],[],[],[],[],[]]
sims_cil = [[],[],[],[],[],[],[]]
for ms in np.unique(cluster_traj_all.compressed()):
    dwell_times_sims = sims_lifetimes_fish[ms]
    dwell_times_real = lifetimes_fish[ms]
    for cf in range(len(dwell_times_real)):
        print(cf)
        m, cil, ciu = stats.bootstrap(ma.hstack(dwell_times_sims[cf]), n_times=100, n_samples=len(ma.hstack(dwell_times_real[cf])))
        hr_sims[ms].append(m)
        sims_cil[ms].append(np.hstack(cil))
        sims_ciu[ms].append(np.hstack(ciu))

In [ ]:
hr_real = [[],[],[],[],[],[],[]]
real_ciu = [[],[],[],[],[],[],[]]
real_cil = [[],[recs_remove],[],[],[]]
for ms in np.unique(cluster_traj_all.compressed()):
    dwell_times_real = lifetimes_fish[ms]
    for cf in range(len(dwell_times_real)):
        print(cf)
        m, cil, ciu = stats.bootstrap(ma.hstack(dwell_times_real[cf]), n_times=100, n_samples=len(ma.hstack(dwell_times_real[cf])))
        hr_real[ms].append(m)
        real_cil[ms].append(np.hstack(cil))
        real_ciu[ms].append(np.hstack(ciu))

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(12.5,12.5))


for ms in [0,1]:
    print(ms)
    ax.scatter(np.asarray(hr_real[ms]),np.asarray(hr_sims[ms]), c = st_colors[ms], s=100, alpha=.6, marker='o')
    ax.errorbar(np.asarray(hr_real[ms]),np.asarray(hr_sims[ms]), 
                yerr=[np.array(hr_sims[ms]) - np.hstack(sims_cil[ms]),np.hstack(sims_ciu[ms]) - np.hstack(hr_sims[ms])], 
                elinewidth=3, capsize=5, fmt='o', color=st_colors[ms], alpha=0.15)
    ax.errorbar(np.asarray(hr_real[ms]),np.asarray(hr_sims[ms]), 
                xerr=[np.array(hr_real[ms]) - np.hstack(real_cil[ms]),np.hstack(real_ciu[ms]) - np.hstack(hr_real[ms])], 
                elinewidth=3, capsize=5, fmt='o', color=st_colors[ms], alpha=0.15)


ax.plot(np.arange(1,1000), np.arange(1,1000), ls = '--', c='k')

ax.set_xscale('log')
ax.set_yscale('log')
plt.xticks(fontsize=35)
plt.yticks(fontsize=35)
# plt.savefig('/Users/gautam.sridhar/Documents/ZENITH/Figures/suppl4/sims_7state_hr_new.pdf')

## Simulations at q=4

In [ ]:
st_colors = ['#343B97', '#00B4F0', '#FBDD2B', '#8D2B24']

In [ ]:
# labels = labels_all

path_to_filtered_data = '/Users/gautam.sridhar/Documents/Repos/ZebraBouts/Results/'
f = h5py.File(path_to_filtered_data + 'pool_ex7_PCs/simlabels_fish_K5_N1200_tau3_cg4.h5')
sims_fishnum = ma.asarray(f['MetaData/simfishes'], dtype=int)
lengths_sims = ma.asarray(f['MetaData/lengths_sims'], dtype=int)
sims_fish = ma.asarray(f['sim_labels'], dtype=int)
f.close()
to_mask = 1300

sims_fish[sims_fish == to_mask] = ma.masked

In [ ]:
kms = np.load('/Users/gautam.sridhar/Documents/Repos/ZebraBouts/Results/pool_ex8_PCs/cg4_labels.npy')

In [ ]:
kmeans_labels = kms

In [ ]:
cluster_traj_all = ma.copy(final_labels)
cluster_traj_all[~final_labels.mask] = ma.array(kmeans_labels)[final_labels[~final_labels.mask]]
cluster_traj_all[final_labels.mask] = ma.masked
cluster_fish = cluster_traj_all.reshape(labels_fish.shape[0],labels_fish.shape[1])
cluster_fish_mask = cluster_fish.mask

In [ ]:
lifetimes_fish = [[],[],[],[],[],[],[]]
n_state = 4

for cf in range(cluster_fish.shape[0]):
    cluster_fish[cf,0] = ma.masked
    cluster_fish[cf,lengths_rem[cf]-1] = ma.masked
    data_lifetimes, state_labels = stats.state_lifetime(cluster_fish[cf,::3],3)
    for k,ks in enumerate(state_labels):
        lifetimes_fish[ks].append(data_lifetimes[k])
    
    if len(state_labels) < n_state:
        exc_state_labels = np.delete(np.arange(n_state),state_labels)
        for k,ks in enumerate(exc_state_labels):
            lifetimes_fish[ks].append([0])

In [ ]:
y_errorbar_metastate, x_all_metastate, m_all_metastate, b_all_metastate, tscales_metastate, params_metastate, mean_lifetimes = stats.state_lifetimes_stats(lifetimes_fish, n_states=2, t0=0, tf=1500 ,n_times=100)

In [ ]:
sims_lifetimes_fish = [[],[],[],[],[],[],[]]
n_state = 2
for cf in range(sims_fish.shape[1]):
    print(cf)
    sf = sims_fish[:,cf,:lengths_rem[cf]].squeeze()
    sf[:,0] = ma.masked
    sf[:,-1] = ma.masked
    sf = ma.concatenate(sims_fish[:,cf,:lengths_rem[cf]].squeeze(), axis=0)
    data_lifetimes, state_labels = stats.state_lifetime(sf,3)
    for k,ks in enumerate(state_labels):
        sims_lifetimes_fish[ks].append(data_lifetimes[k])
        
    if len(state_labels) < n_state:
        print(len(state_labels))
        exc_state_labels = np.delete(np.arange(n_state),state_labels)
        for k,ks in enumerate(exc_state_labels):
            sims_lifetimes_fish[ks].append([0])

In [ ]:
y_errorbar_metastate_sims, x_all_metastate_sims, m_all_metastate, b_all_metastate, tscales_metastate, params_metastate, mean_lifetimes = stats.state_lifetimes_stats(sims_lifetimes_fish, n_states=2,t0=0,tf=1500, n_times=100)

In [ ]:
fig,ax = plt.subplots(1,1, figsize=(12.5,12.5))

t0,tf = 1,600

xrange = np.linspace(t0,tf,160)

sim_markers = ['o','s']

# st_names = ['Metastate 1', 'Metastate 2']

# for state_idx in np.unique(kmeans_labels):
#     x,y = stats.cumulative_dist(data_f_lt[state_idx],[0,1000])
#     y = np.array([np.mean(y[x==x_unique]) for x_u nique in np.unique(x)])
#     x = np.sort(np.unique(x))
#     plt.plot(x,1-y,lw=2,ls='--',c=st_colors[state_idx])

for ms in [0,1]:#np.unique(cluster_traj_all.compressed()):

    mean = y_errorbar_metastate[ms][:-1,0]
    cil = y_errorbar_metastate[ms][:-1,1]
    ciu = y_errorbar_metastate[ms][:-1,2]
    
    mean_sim = y_errorbar_metastate_sims[ms][:-1,0]
    cil_sim = y_errorbar_metastate_sims[ms][:-1,1]
    ciu_sim = y_errorbar_metastate_sims[ms][:-1,2]
    
    ax.scatter(x_all_metastate[ms][:-1], mean, marker='^',s=50, color='C{}'.format(ms))#,label = r'Mean Duration$_{} = {}$ Bouts'.format({st_names2[ms][0:2]}, round(mean_lifetimes[ms],2)))
    ax.errorbar(x_all_metastate[ms][:-1], mean,[mean-cil, ciu - mean], fmt='.', capsize=10, color = 'C{}'.format(ms), alpha=0.4)

    ax.scatter(x_all_metastate_sims[ms][:-1], mean_sim,s=100, marker='o',facecolors='none', edgecolors='C{}'.format(ms))
    ax.fill_between(x_all_metastate_sims[ms][:-1], cil_sim, ciu_sim, alpha=0.3, color='C{}'.format(ms))
    
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.set_yscale('log')
# ax.set_title('Actual and Simulated dwell times', fontsize=35)
ax.set_xscale('log')
ax.set_xlabel(r'Number of states',fontsize=40)
ax.set_ylabel(r'Complementary CDF (log scale)',fontsize= 40)
ax.legend(loc="upper left",bbox_to_anchor=(0.4, 0.96), fontsize=30)
plt.xticks(fontsize=30)
plt.yticks(fontsize=30)
# ax.set_xlim(1,1000)
# ax.set_ylim(5e-6,1) 

# plt.savefig('/Users/gautam.sridhar/Documents/ZENITH/Figures/Suppl4/2state_dwelltime_dist.pdf')
plt.show()

In [ ]:
hr_sims = [[],[],[],[],[],[],[]]
sims_ciu = [[],[],[],[],[],[],[]]
sims_cil = [[],[],[],[],[],[],[]]
for ms in np.unique(cluster_traj_all.compressed()):
    dwell_times_sims = sims_lifetimes_fish[ms]
    dwell_times_real = lifetimes_fish[ms]
    for cf in range(len(dwell_times_real)):
        print(cf)
        m, cil, ciu = stats.bootstrap(ma.hstack(dwell_times_sims[cf]), n_times=100, n_samples=len(ma.hstack(dwell_times_real[cf])))
        hr_sims[ms].append(m)
        sims_cil[ms].append(np.hstack(cil))
        sims_ciu[ms].append(np.hstack(ciu))

In [ ]:
hr_real = [[],[],[],[],[],[],[]]
real_ciu = [[],[],[],[],[],[],[]]
real_cil = [[],[recs_remove],[],[],[]]
for ms in np.unique(cluster_traj_all.compressed()):
    dwell_times_real = lifetimes_fish[ms]
    for cf in range(len(dwell_times_real)):
        print(cf)
        m, cil, ciu = stats.bootstrap(ma.hstack(dwell_times_real[cf]), n_times=100, n_samples=len(ma.hstack(dwell_times_real[cf])))
        hr_real[ms].append(m)
        real_cil[ms].append(np.hstack(cil))
        real_ciu[ms].append(np.hstack(ciu))

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(12.5,12.5))


for ms in [0,1]:
    print(ms)
    ax.scatter(np.asarray(hr_real[ms]),np.asarray(hr_sims[ms]), c = st_colors[ms], s=100, alpha=.6, marker='o')
    ax.errorbar(np.asarray(hr_real[ms]),np.asarray(hr_sims[ms]), 
                yerr=[np.array(hr_sims[ms]) - np.hstack(sims_cil[ms]),np.hstack(sims_ciu[ms]) - np.hstack(hr_sims[ms])], 
                elinewidth=3, capsize=5, fmt='o', color=st_colors[ms], alpha=0.15)
    ax.errorbar(np.asarray(hr_real[ms]),np.asarray(hr_sims[ms]), 
                xerr=[np.array(hr_real[ms]) - np.hstack(real_cil[ms]),np.hstack(real_ciu[ms]) - np.hstack(hr_real[ms])], 
                elinewidth=3, capsize=5, fmt='o', color=st_colors[ms], alpha=0.15)


ax.plot(np.arange(1,1000), np.arange(1,1000), ls = '--', c='k')

ax.set_xscale('log')
ax.set_yscale('log')
plt.xticks(fontsize=35)
plt.yticks(fontsize=35)
# plt.savefig('/Users/gautam.sridhar/Documents/ZENITH/Figures/suppl4/sims_7state_hr_new.pdf')

## Simulations at q=7

In [ ]:
st_colors = ['#006FB8','#313390','#52C1AF','#00B4F0','#F4BD48','#FBDD2B','#8D2B24']

In [ ]:
# labels = labels_all

path_to_filtered_data = '/Users/gautam.sridhar/Documents/Repos/ZebraBouts/Results/'
f = h5py.File(path_to_filtered_data + 'pool_ex7_PCs/simlabels_fish_K5_N1200_tau3_cg7.h5')
sims_fishnum = ma.asarray(f['MetaData/simfishes'], dtype=int)
lengths_sims = ma.asarray(f['MetaData/lengths_sims'], dtype=int)
sims_fish = ma.asarray(f['sim_labels'], dtype=int)
f.close()
to_mask = 1300

sims_fish[sims_fish == to_mask] = ma.masked

In [ ]:
kms = np.load('/Users/gautam.sridhar/Documents/Repos/ZebraBouts/Results/pool_ex8_PCs/cg7_labels.npy')

In [ ]:
kmeans_labels = kms

In [ ]:
cluster_traj_all = ma.copy(final_labels)
cluster_traj_all[~final_labels.mask] = ma.array(kmeans_labels)[final_labels[~final_labels.mask]]
cluster_traj_all[final_labels.mask] = ma.masked
cluster_fish = cluster_traj_all.reshape(labels_fish.shape[0],labels_fish.shape[1])
cluster_fish_mask = cluster_fish.mask

In [ ]:
lifetimes_fish = [[],[],[],[],[],[],[]]
n_state = 4

for cf in range(cluster_fish.shape[0]):
    cluster_fish[cf,0] = ma.masked
    cluster_fish[cf,lengths_rem[cf]-1] = ma.masked
    data_lifetimes, state_labels = stats.state_lifetime(cluster_fish[cf,::3],3)
    for k,ks in enumerate(state_labels):
        lifetimes_fish[ks].append(data_lifetimes[k])
    
    if len(state_labels) < n_state:
        exc_state_labels = np.delete(np.arange(n_state),state_labels)
        for k,ks in enumerate(exc_state_labels):
            lifetimes_fish[ks].append([0])

In [ ]:
y_errorbar_metastate, x_all_metastate, m_all_metastate, b_all_metastate, tscales_metastate, params_metastate, mean_lifetimes = stats.state_lifetimes_stats(lifetimes_fish, n_states=2, t0=0, tf=1500 ,n_times=100)

In [ ]:
sims_lifetimes_fish = [[],[],[],[],[],[],[]]
n_state = 2
for cf in range(sims_fish.shape[1]):
    print(cf)
    sf = sims_fish[:,cf,:lengths_rem[cf]].squeeze()
    sf[:,0] = ma.masked
    sf[:,-1] = ma.masked
    sf = ma.concatenate(sims_fish[:,cf,:lengths_rem[cf]].squeeze(), axis=0)
    data_lifetimes, state_labels = stats.state_lifetime(sf,3)
    for k,ks in enumerate(state_labels):
        sims_lifetimes_fish[ks].append(data_lifetimes[k])
        
    if len(state_labels) < n_state:
        print(len(state_labels))
        exc_state_labels = np.delete(np.arange(n_state),state_labels)
        for k,ks in enumerate(exc_state_labels):
            sims_lifetimes_fish[ks].append([0])

In [ ]:
y_errorbar_metastate_sims, x_all_metastate_sims, m_all_metastate, b_all_metastate, tscales_metastate, params_metastate, mean_lifetimes = stats.state_lifetimes_stats(sims_lifetimes_fish, n_states=2,t0=0,tf=1500, n_times=100)

In [ ]:
fig,ax = plt.subplots(1,1, figsize=(12.5,12.5))

t0,tf = 1,600

xrange = np.linspace(t0,tf,160)

sim_markers = ['o','s']

# st_names = ['Metastate 1', 'Metastate 2']

# for state_idx in np.unique(kmeans_labels):
#     x,y = stats.cumulative_dist(data_f_lt[state_idx],[0,1000])
#     y = np.array([np.mean(y[x==x_unique]) for x_u nique in np.unique(x)])
#     x = np.sort(np.unique(x))
#     plt.plot(x,1-y,lw=2,ls='--',c=st_colors[state_idx])

for ms in [0,1]:#np.unique(cluster_traj_all.compressed()):

    mean = y_errorbar_metastate[ms][:-1,0]
    cil = y_errorbar_metastate[ms][:-1,1]
    ciu = y_errorbar_metastate[ms][:-1,2]
    
    mean_sim = y_errorbar_metastate_sims[ms][:-1,0]
    cil_sim = y_errorbar_metastate_sims[ms][:-1,1]
    ciu_sim = y_errorbar_metastate_sims[ms][:-1,2]
    
    ax.scatter(x_all_metastate[ms][:-1], mean, marker='^',s=50, color='C{}'.format(ms))#,label = r'Mean Duration$_{} = {}$ Bouts'.format({st_names2[ms][0:2]}, round(mean_lifetimes[ms],2)))
    ax.errorbar(x_all_metastate[ms][:-1], mean,[mean-cil, ciu - mean], fmt='.', capsize=10, color = 'C{}'.format(ms), alpha=0.4)

    ax.scatter(x_all_metastate_sims[ms][:-1], mean_sim,s=100, marker='o',facecolors='none', edgecolors='C{}'.format(ms))
    ax.fill_between(x_all_metastate_sims[ms][:-1], cil_sim, ciu_sim, alpha=0.3, color='C{}'.format(ms))
    
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.set_yscale('log')
# ax.set_title('Actual and Simulated dwell times', fontsize=35)
ax.set_xscale('log')
ax.set_xlabel(r'Number of states',fontsize=40)
ax.set_ylabel(r'Complementary CDF (log scale)',fontsize= 40)
ax.legend(loc="upper left",bbox_to_anchor=(0.4, 0.96), fontsize=30)
plt.xticks(fontsize=30)
plt.yticks(fontsize=30)
# ax.set_xlim(1,1000)
# ax.set_ylim(5e-6,1) 

# plt.savefig('/Users/gautam.sridhar/Documents/ZENITH/Figures/Suppl4/2state_dwelltime_dist.pdf')
plt.show()

In [ ]:
hr_sims = [[],[],[],[],[],[],[]]
sims_ciu = [[],[],[],[],[],[],[]]
sims_cil = [[],[],[],[],[],[],[]]
for ms in np.unique(cluster_traj_all.compressed()):
    dwell_times_sims = sims_lifetimes_fish[ms]
    dwell_times_real = lifetimes_fish[ms]
    for cf in range(len(dwell_times_real)):
        print(cf)
        m, cil, ciu = stats.bootstrap(ma.hstack(dwell_times_sims[cf]), n_times=100, n_samples=len(ma.hstack(dwell_times_real[cf])))
        hr_sims[ms].append(m)
        sims_cil[ms].append(np.hstack(cil))
        sims_ciu[ms].append(np.hstack(ciu))

In [ ]:
hr_real = [[],[],[],[],[],[],[]]
real_ciu = [[],[],[],[],[],[],[]]
real_cil = [[],[recs_remove],[],[],[]]
for ms in np.unique(cluster_traj_all.compressed()):
    dwell_times_real = lifetimes_fish[ms]
    for cf in range(len(dwell_times_real)):
        print(cf)
        m, cil, ciu = stats.bootstrap(ma.hstack(dwell_times_real[cf]), n_times=100, n_samples=len(ma.hstack(dwell_times_real[cf])))
        hr_real[ms].append(m)
        real_cil[ms].append(np.hstack(cil))
        real_ciu[ms].append(np.hstack(ciu))

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(12.5,12.5))


for ms in [0,1]:
    print(ms)
    ax.scatter(np.asarray(hr_real[ms]),np.asarray(hr_sims[ms]), c = st_colors[ms], s=100, alpha=.6, marker='o')
    ax.errorbar(np.asarray(hr_real[ms]),np.asarray(hr_sims[ms]), 
                yerr=[np.array(hr_sims[ms]) - np.hstack(sims_cil[ms]),np.hstack(sims_ciu[ms]) - np.hstack(hr_sims[ms])], 
                elinewidth=3, capsize=5, fmt='o', color=st_colors[ms], alpha=0.15)
    ax.errorbar(np.asarray(hr_real[ms]),np.asarray(hr_sims[ms]), 
                xerr=[np.array(hr_real[ms]) - np.hstack(real_cil[ms]),np.hstack(real_ciu[ms]) - np.hstack(hr_real[ms])], 
                elinewidth=3, capsize=5, fmt='o', color=st_colors[ms], alpha=0.15)


ax.plot(np.arange(1,1000), np.arange(1,1000), ls = '--', c='k')

ax.set_xscale('log')
ax.set_yscale('log')
plt.xticks(fontsize=35)
plt.yticks(fontsize=35)
# plt.savefig('/Users/gautam.sridhar/Documents/ZENITH/Figures/suppl4/sims_7state_hr_new.pdf')